In [2]:
# LTN: https://github.com/logictensornetworks/logictensornetworks/blob/master/examples/multiclass_classification/multiclass-singlelabel.ipynb
# Common.py : https://github.com/logictensornetworks/logictensornetworks/raw/master/examples/multiclass_classification/commons.py

In [3]:
!pip install PyTDC rdkit-pypi ltn keras==2.15.0 -qq
!wget https://github.com/logictensornetworks/logictensornetworks/raw/master/examples/multiclass_classification/commons.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.3/151.3 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import logging; logging.basicConfig(level=logging.INFO)
import tensorflow as tf
import pandas as pd
import numpy as np
import ltn
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm.auto import tqdm
tqdm.pandas()

In [7]:
# Threshold ref: https://pubs.acs.org/doi/epdf/10.1021/acs.jcim.3c01301
def label_th(pic50):
    classes = []
    for x in pic50:
        if x>=5:
            classes.append(1)
        else:
            classes.append(0)

    return np.asarray(classes)

class_map = {
    "blocks":1,
    "non-blocks":0,
}

### Data Acquisition

In [8]:
dataset_path = "/content/drive/MyDrive/Project/AI and Cardiology/Cardiotoxicity/Dataset"

In [9]:
!ls "{dataset_path}/UniChemDB-Data"

CDK-unichemdb.csv     herg_uniherg_db_mmb_emb.npy  uniherg_db-deepseek-qwen1_5b-embedding.parquet
final-herg.csv	      Morgan-unichemdb.csv
final-herg-split.csv  standized-herg.csv


In [10]:
df = pd.read_parquet(f"{dataset_path}/UniChemDB-Data/uniherg_db-deepseek-qwen1_5b-embedding.parquet")

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20409 entries, 0 to 20408
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                20388 non-null  object
 1   std_smiles        20409 non-null  object
 2   classes           20409 non-null  int64 
 3   train_test_split  20409 non-null  int64 
 4   cv_fold           20409 non-null  int64 
 5   Embeddings        20409 non-null  object
dtypes: int64(3), object(3)
memory usage: 956.8+ KB


# Data



In [12]:
#External Test-1: https://github.com/Abdulk084/CardioTox/blob/master/data/external_test_set_pos.csv
ext_pos_df = pd.read_parquet(f"{dataset_path}/External-Data/pos-uniherg_db-deepseek-qwen1_5b-embedding.parquet")

In [13]:
# External Test h70, h60 dataset: https://github.com/issararab/CToxPred/tree/main/data/raw/hERG
ext_h60_df = pd.read_parquet(f"{dataset_path}/External-Data/h60-uniherg_db-deepseek-qwen1_5b-embedding.parquet")
ext_h70_df = pd.read_parquet(f"{dataset_path}/External-Data/h70-uniherg_db-deepseek-qwen1_5b-embedding.parquet")

In [14]:
ext_h60_df.head()

,ACTIVITY,smiles,emb
0,0,CCOC(=O)C1(CCN(C)CC1)c1ccccc1,"[-0.3822215, -0.42416444, 0.32713515, -0.30706..."
1,0,CCN(CC)CC(=O)NC1=C(C)C=CC=C1C,"[-0.2667105, 0.018063817, -0.28471446, 0.18878..."
2,0,CCCC(CCC)C(=O)O,"[-0.5838299, 0.259647, -0.333125, -1.5202333, ..."
3,0,CCC(COC(=O)c1cc(OC)c(OC)c(OC)c1)(c1ccccc1)N(C)C,"[-0.32278958, -0.46425068, 0.6418327, -0.43008..."
4,0,COc1ccc(N(C(C)=O)c2cc3c(cc2[N+](=O)[O-])OC(C)(...,"[-1.0382861, -0.46823546, 1.0130138, -0.178561..."


In [15]:
!ls '{dataset_path}/External-Data'

CDK-external_test_set_pos.csv
CDK-herg60.csv
CDK-herg70.csv
eval_set_herg_60.csv
eval_set_herg_70.csv
external_test_set_pos.csv
h60-uniherg_db-deepseek-qwen1_5b-embedding.parquet
h70-uniherg_db-deepseek-qwen1_5b-embedding.parquet
herg_mmb_emb_external_test.npz
herg_mmb_emb_h60.npz
herg_mmb_emb_h70.npz
Morgan-external_test_set_pos.csv
Morgan-herg60.csv
Morgan-herg70.csv
pos-uniherg_db-deepseek-qwen1_5b-embedding.parquet


In [16]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X = ss.fit_transform(np.vstack(df['Embeddings'].values) )
y = df['classes']


In [17]:
X.shape,y.shape

((20409, 1536), (20409,))

In [18]:
batch_size = 64
ds_train = tf.data.Dataset.from_tensor_slices((X,y)).batch(batch_size)
idx = np.random.random_integers(0,len(X),1000)
ds_test = tf.data.Dataset.from_tensor_slices((X[idx],y[idx])).batch(batch_size)

<ipython-input-18-21e067a5c8e3>:3: DeprecationWarning: This function is deprecated. Please call randint(0, 20409 + 1) instead
  idx = np.random.random_integers(0,len(X),1000)


# LTN

Predicate with softmax `P(x,class)`

In [19]:
class MLP(tf.keras.Model):
    """Model that returns logits."""
    def __init__(self, n_classes, hidden_layer_sizes=(16,16,8)):
        super(MLP, self).__init__()
        self.denses = [tf.keras.layers.Dense(s, activation="elu") for s in hidden_layer_sizes]
        self.dense_class = tf.keras.layers.Dense(n_classes)
        self.dropout = tf.keras.layers.Dropout(0.2)

    def call(self, inputs, training=False):
        x = inputs[0]
        for dense in self.denses:
            x = dense(x)
            x = self.dropout(x, training=training)
        return self.dense_class(x)

logits_model = MLP(2)
p = ltn.Predicate.FromLogits(logits_model, activation_function="softmax", with_class_indexing=True)

Constants to index/iterate on the classes

In [20]:
class_A = ltn.Constant(0, trainable=False)
class_B = ltn.Constant(1, trainable=False)
# class_C = ltn.Constant(2, trainable=False)

Operators and axioms

In [21]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")

In [22]:
formula_aggregator = ltn.Wrapper_Formula_Aggregator(ltn.fuzzy_ops.Aggreg_pMeanError(p=2))

@tf.function
def axioms(features, labels, training=False):
    x_A = ltn.Variable("x_A",features[labels==0])
    x_B = ltn.Variable("x_B",features[labels==1])
    # x_C = ltn.Variable("x_C",features[labels==2])
    axioms = [
        Forall(x_A,p([x_A,class_A],training=training)),
        Forall(x_B,p([x_B,class_B],training=training)),
        # Forall(x_C,p([x_C,class_C],training=training))
    ]
    for i in range(len(axioms)):
        if tf.math.is_nan(axioms[i].tensor):
            axioms[i].tensor  =0.0
    sat_level = formula_aggregator(axioms).tensor
    return sat_level

Initialize all layers and the static graph

In [23]:
for features, labels in ds_test:
    print("Initial sat level %.5f"%axioms(features,labels))
    break

Initial sat level 0.42535


# Training

Define the metrics. While training, we measure:
1. The level of satisfiability of the Knowledge Base of the training data.
1. The level of satisfiability of the Knowledge Base of the test data.
3. The training accuracy.
4. The test accuracy.

In [24]:
metrics_dict = {
    'train_sat_kb': tf.keras.metrics.Mean(name='train_sat_kb'),
    'test_sat_kb': tf.keras.metrics.Mean(name='test_sat_kb'),
    'train_accuracy': tf.keras.metrics.CategoricalAccuracy(name="train_accuracy"),
    'test_accuracy': tf.keras.metrics.CategoricalAccuracy(name="test_accuracy")
}

Define the training and test step

In [25]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
@tf.function
def train_step(features, labels):
    # sat and update
    with tf.GradientTape() as tape:
        sat = axioms(features, labels, training=True)
        loss = 1.-sat
    gradients = tape.gradient(loss, p.trainable_variables)
    optimizer.apply_gradients(zip(gradients, p.trainable_variables))
    sat = axioms(features, labels) # compute sat without dropout
    metrics_dict['train_sat_kb'](sat)
    # accuracy
    predictions = logits_model([features])
    metrics_dict['train_accuracy'](tf.one_hot(labels,2),predictions)

@tf.function
def test_step(features, labels):
    # sat
    sat = axioms(features, labels)
    metrics_dict['test_sat_kb'](sat)
    # accuracy
    predictions = logits_model([features])
    metrics_dict['test_accuracy'](tf.one_hot(labels,2),predictions)

Train

In [26]:
import commons

EPOCHS = 500

commons.train(
    EPOCHS,
    metrics_dict,
    ds_train,
    ds_test,
    train_step,
    test_step,
    csv_path="herg_deepseek_results.csv",
    track_metrics=20
)

Epoch 0, train_sat_kb: 0.4756, test_sat_kb: 0.5347, train_accuracy: 0.6248, test_accuracy: 0.6860
Epoch 20, train_sat_kb: 0.5810, test_sat_kb: 0.6040, train_accuracy: 0.7897, test_accuracy: 0.7900
Epoch 40, train_sat_kb: 0.6203, test_sat_kb: 0.6378, train_accuracy: 0.8332, test_accuracy: 0.8280
Epoch 60, train_sat_kb: 0.6487, test_sat_kb: 0.6649, train_accuracy: 0.8586, test_accuracy: 0.8550
Epoch 80, train_sat_kb: 0.6662, test_sat_kb: 0.6777, train_accuracy: 0.8774, test_accuracy: 0.8690
Epoch 100, train_sat_kb: 0.6796, test_sat_kb: 0.6794, train_accuracy: 0.8871, test_accuracy: 0.8730
Epoch 120, train_sat_kb: 0.6902, test_sat_kb: 0.6892, train_accuracy: 0.8948, test_accuracy: 0.8790
Epoch 140, train_sat_kb: 0.6994, test_sat_kb: 0.7043, train_accuracy: 0.9019, test_accuracy: 0.8910
Epoch 160, train_sat_kb: 0.7095, test_sat_kb: 0.7156, train_accuracy: 0.9072, test_accuracy: 0.8930
Epoch 180, train_sat_kb: 0.7153, test_sat_kb: 0.7185, train_accuracy: 0.9133, test_accuracy: 0.8990
Epoch 

In [27]:
!ls "{dataset_path}/../Model-Weights"

hERG-Karim-CDK.keras	     hERG-Karim-Morgan.keras	 hERG-UniChemDB-MMB.keras
hERG-Karim-MMB.keras	     hERG-UniChemDB-CDK.keras	 hERG-UniChemDB-Morgan_CDK.keras
hERG-Karim-Morgan_CDK.keras  hERG-UniChemDB-LLAMA.keras  hERG-UniChemDB-Morgan.keras


In [28]:
logits_model.save(f"{dataset_path}/../Model-Weights/hERG-UniChemDB-DeepSeek.keras")

## Model Evaluation

In [29]:
from sklearn.metrics import (
    accuracy_score as ays,
    f1_score as fs,
    precision_score as ps,
    recall_score as rs,
    matthews_corrcoef as mcc,
    roc_auc_score as auc,
    balanced_accuracy_score,
    confusion_matrix
)

In [30]:
def print_score(xtest,ytest,name):

    pred_test = logits_model.predict([xtest]).argmax(-1)

    auc_test = auc(ytest, pred_test)


    tn, fp, fn, tp = confusion_matrix(ytest, pred_test).ravel()

    specificity_test = tn / (tn + fp)

    sensitivity_test = tp / (tp + fn)

    NPV_test = tn / (tn + fn)

    PPV_test = tp / (tp + fp)
    Accuracy_test = ays(ytest, pred_test)
    Balanced_Accuracy_test = balanced_accuracy_score(ytest, pred_test)

    MCC_test= mcc(ytest, pred_test)


    print(f"MCC_test_{name}: " + str(MCC_test))
    print(f"NPV_test_{name}g: " + str(NPV_test))
    print(f"Accuracy_test_{name}: " + str(Accuracy_test))
    print(f"PPV_test_{name}: " + str(PPV_test))
    print(f"specificity_test_{name}: " + str(specificity_test))
    print(f"sensitivity_test_{name}: " + str(sensitivity_test))
    print(f"Balanced_Accuracy_test{name}: " + str(Balanced_Accuracy_test))


In [31]:
print_score(ss.transform(np.vstack(ext_pos_df['emb'].values)),ext_pos_df['ACTIVITY'],'External Data Test-1 (pos)')

2/2 [==============================] - 0s 5ms/step
MCC_test_External Data Test-1 (pos): 0.6850937041446569
NPV_test_External Data Test-1 (pos)g: 0.6842105263157895
Accuracy_test_External Data Test-1 (pos): 0.8409090909090909
PPV_test_External Data Test-1 (pos): 0.96
specificity_test_External Data Test-1 (pos): 0.9285714285714286
sensitivity_test_External Data Test-1 (pos): 0.8
Balanced_Accuracy_testExternal Data Test-1 (pos): 0.8642857142857143


In [36]:
print_score(ss.transform(np.vstack(ext_h60_df['emb'].values)),(ext_h60_df.ACTIVITY).astype(int),'External hERG-60')


2/2 [==============================] - 0s 8ms/step
MCC_test_External hERG-60: 0.6850937041446569
NPV_test_External hERG-60g: 0.6842105263157895
Accuracy_test_External hERG-60: 0.8409090909090909
PPV_test_External hERG-60: 0.96
specificity_test_External hERG-60: 0.9285714285714286
sensitivity_test_External hERG-60: 0.8
Balanced_Accuracy_testExternal hERG-60: 0.8642857142857143


In [33]:
print_score(ss.transform(np.vstack(ext_h70_df['emb'].values)),(ext_h70_df.pIC50 >=5).astype(int),'External hERG-70')


15/15 [==============================] - 0s 2ms/step
MCC_test_External hERG-70: 0.5871242081525284
NPV_test_External hERG-70g: 0.7125984251968503
Accuracy_test_External hERG-70: 0.7864693446088795
PPV_test_External hERG-70: 0.8721461187214612
specificity_test_External hERG-70: 0.8660287081339713
sensitivity_test_External hERG-70: 0.7234848484848485
Balanced_Accuracy_testExternal hERG-70: 0.7947567783094098
